In [21]:
from dataclasses import dataclass
import itertools
from typing import Any, Dict, Sequence, Union


NAME_DELIMITER = '-'
DISMISS_FORM = object()


def name(*parts: str) -> str:
    return NAME_DELIMITER.join(parts)


def dename(name: str) -> Sequence[str]:
    return name.split(NAME_DELIMITER)


class Form:
    NORMAL = '__normal__'
    # normal is implicit as there currently is no non-normal female form
    # with significant differences
    FEMALE = 'female'
    
    MEGA = 'mega'
    MEGA_X = name('mega', 'x')
    MEGA_Y = name('mega', 'y')
    GIGANTAMAX = 'gigantamax'
    ALOLA = 'alola'
    GALAR = 'galar'
    HISUI = 'hisui'


@dataclass
class PokemonForm:
    """Specifies an appearance of a pokemon."""
    
    ndex: int
    name: str
    color_only: bool = False

    @property
    def complete_name(self):
        if self.name == Form.NORMAL:
            return f'{self.ndex}'
        else:
            return name(str(self.ndex), self.name)


class NoNormal(tuple):
    ...


def to_kwargs(form_descriptor: Union[str, Dict[str, Any], ]):
    """Converts a form descriptor to kwargs suitable for a PokemonForm instance."""
    if isinstance(form_descriptor, str):
        kwargs = dict(name=form_descriptor)
    elif isinstance(form_descriptor, dict):
        assert all(isinstance(key, str) for key in form_descriptor.keys()), (
            'dict with non-string keys cannot be used as kwargs'
        )
        kwargs = form_descriptor
    return kwargs


def get_instances(ndex, form_descriptors: Union[str, tuple]):
    if not isinstance(form_descriptors, tuple):
        form_descriptors = (form_descriptors,)
    
    if not isinstance(form_descriptors, NoNormal):
        form_descriptors = (Form.NORMAL, *form_descriptors)

    return tuple(
        PokemonForm(ndex=ndex, **to_kwargs(form)) 
        for form in form_descriptors
    )
            


def get_forms(forms_by_ndex):
    default_forms = {
        ndex: (PokemonForm(ndex=ndex, name=Form.NORMAL), )
        for ndex in range(1, max(forms_by_ndex.keys()) + 1)
    }
    return {
        **default_forms,
        **{
            ndex: get_instances(ndex, forms)
            for ndex, forms in forms_by_ndex.items()
        },
    }


def get_form(ndex: int, form_name: str) -> PokemonForm:
    forms = POKEMON_FORMS[ndex]
    matches = [f for f in forms if f.name == form_name]
    assert len(matches) == 1, f'got {len(matches)} matching forms instead 1 for #{name(str(ndex), form_name)}'
    return matches[0]


def no_normal(*forms):
    return NoNormal(forms)


def color_only(*forms):
    return tuple(
        {**to_kwargs(form), 'color_only': True}
        for form in forms
    )


# https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_with_form_differences
POKEMON_FORMS = get_forms({
    3: (Form.MEGA, Form.GIGANTAMAX),  # Venusaur
    6: (Form.MEGA_X, Form.MEGA_Y, Form.GIGANTAMAX),  # Charizard
    9: (Form.MEGA, Form.GIGANTAMAX),  # Blastoise
    12: Form.GIGANTAMAX,  # Butterfree
    15: Form.MEGA,  # Beedrill
    18: Form.MEGA,  # Pidgeot
    19: Form.ALOLA,  # Rattata
    20: Form.ALOLA,  # Raticate
    25: (
        Form.GIGANTAMAX,
        'cosplay', 'cosplay-rock-star', 'cosplay-belle', 
        'cosplay-pop-star', 'cosplay-phd', 'cosplay-libre',
        'cap-original', 'cap-hoenn', 'cap-sinnoh', 'cap-unova', 
        'cap-kalos', 'cap-alola', 'cap-partner', 'cap-world',
    ),  # Pikachu
    26: Form.ALOLA,  # Raichu
    27: Form.ALOLA,  # Sandshrew
    28: Form.ALOLA,  # Sandslash
    37: Form.ALOLA,  # Vulpix
    38: Form.ALOLA,  # Ninetales
    50: Form.ALOLA,  # Diglett
    51: Form.ALOLA,  # Dugtrio
    52: (Form.ALOLA, Form.GALAR, Form.GIGANTAMAX),  # Meowth
    53: Form.ALOLA,  # Persian
    58: Form.HISUI,  # Growlithe
    65: Form.MEGA,  # Alakazam
    68: Form.GIGANTAMAX,  # Machamp
    74: Form.ALOLA,  # Geodude
    75: Form.ALOLA,  # Graveler
    76: Form.ALOLA,  # Golem
    77: Form.GALAR,  # Ponyta
    78: Form.GALAR,  # Rapidash
    79: Form.GALAR,  # Slowpoke
    80: (Form.MEGA, Form.GALAR),  # Slowbro
    83: Form.GALAR,  # Farfetch'd
    88: Form.ALOLA,  # Grimer
    89: Form.ALOLA,  # Muk
    94: (Form.MEGA, Form.GIGANTAMAX),  # Gengar
    99: Form.GIGANTAMAX,  # Kingler
    103: Form.ALOLA,  # Exeggutor
    105: Form.ALOLA,  # Marowak
    110: Form.GALAR,  # Weezing
    115: Form.MEGA,  # Kangaskhan
    122: Form.GALAR,  # Mr. Mime
    127: Form.MEGA,  # Pinsir
    130: Form.MEGA,  # Gyarados
    131: Form.GIGANTAMAX,  # Lapras
    133: Form.GIGANTAMAX,  # Eevee
    142: Form.MEGA,  # Aerodactyl
    143: Form.GIGANTAMAX,  # Snorlax
    144: Form.GALAR,  # Articuno
    145: Form.GALAR,  # Zapdos
    146: Form.GALAR,  # Moltres
    150: (Form.MEGA_X, Form.MEGA_Y),  # Mewtwo
    172: 'spiky-eared',  # Pichu
    181: Form.MEGA,  # Ampharos
    199: Form.GALAR,  # Slowking
    201: no_normal(
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 
        'h', 'i', 'j', 'k', 'l', 'm', 'n', 
        'o', 'p', 'q', 'r', 's', 't', 'u', 
        'v', 'w', 'x', 'y', 'z',
        'exclamation', 'question',
    ),  # Unown
    208: Form.MEGA,  # Steelix
    212: Form.MEGA,  # Scizor
    214: Form.MEGA,  # Heracross
    222: Form.GALAR,  # Corsola
    229: Form.MEGA,  # Houndoom
    248: Form.MEGA,  # Tyranitar
    254: Form.MEGA,  # Sceptile
    257: Form.MEGA,  # Blaziken
    260: Form.MEGA,  # Swampert
    263: Form.GALAR,  # Zigzagoon
    264: Form.GALAR,  # Linoone
    282: Form.MEGA,  # Gardevoir
    302: Form.MEGA,  # Sableye
    303: Form.MEGA,  # Mawile
    306: Form.MEGA,  # Aggron
    308: Form.MEGA,  # Medicham
    310: Form.MEGA,  # Manectric
    319: Form.MEGA,  # Sharpedo
    323: Form.MEGA,  # Camerupt
    334: Form.MEGA,  # Altaria
    351: ('sunny', 'rainy', 'snowy'),  # Castform
    354: Form.MEGA,  # Banette
    359: Form.MEGA,  # Absol
    362: Form.MEGA,  # Glalie
    373: Form.MEGA,  # Salamence
    376: Form.MEGA,  # Metagross
    380: Form.MEGA,  # Latias
    381: Form.MEGA,  # Latios
    382: 'primal',  # Kyogre
    383: 'primal',  # Groudon
    384: Form.MEGA,  # Rayquaza
    386: ('attack', 'defense', 'speed'),  # Deoxys
    412: no_normal('plant', 'sandy', 'trash'),  # Burmy
    413: no_normal('plant', 'sandy', 'trash'),  # Wormadam
    421: no_normal('overcast', 'sunshine'),  # Cherrim
    422: no_normal('east', 'west'),  # Shellos
    423: no_normal('east', 'west'),  # Gastrodon
    428: Form.MEGA,  # Lopunny
    445: Form.MEGA,  # Garchomp
    448: Form.MEGA,  # Lucario
    460: Form.MEGA,  # Abomasnow
    475: Form.MEGA,  # Gallade
    479: ('fan', 'frost', 'heat', 'mow', 'wash'),  # Rotom
    487: no_normal('altered', 'origin'),  # Giratina
    492: no_normal('land', 'sky'),  # Shaymin
    # TODO: COLOR ONLY
    493: (
        'bug', 'dark', 'dragon', 'electric', 'fighting', 'fire', 
        'flying', 'ghost', 'grass', 'ground', 'ice',  # 'normal', 
        'poison', 'psychic', 'rock', 'steel', 'water',
    ),  # Arceus
    521: Form.FEMALE,  # Unfezant
    531: Form.MEGA,  # Audino
    550: no_normal('blue-striped', 'red-striped'),  # Basculin
    554: Form.GALAR,  # Darumaka
    555: ('zen', Form.GALAR, f'{Form.GALAR}-zen'),  # Darmanitan
    569: Form.GIGANTAMAX,  # Garbodor
    # TODO: COLOR ONLY
    585: no_normal('spring', 'summer', 'autumn', 'winter'),  # Deerling
    586: no_normal('spring', 'summer', 'autumn', 'winter'),  # Sawsbuck
    592: Form.FEMALE,  # Frillish
    593: Form.FEMALE,  # Jellicent
    628: Form.HISUI,  # Braviary
    641: no_normal('incarnate', 'therian'),  # Tornadus
    642: no_normal('incarnate', 'therian'),  # Thundurus
    643: 'overdrive',  # Reshiram
    644: 'overdrive',  # Zekrom
    645: no_normal('incarnate', 'therian'),  # Landorus
    646: (
        'black', 'black-overdrive',
        'white', 'white-overdrive',
    ),  # Kyurem
    647: no_normal('ordinary', 'resolute'),  # Keldeo
    648: no_normal('aria', 'pirouette'),  # Meloetta
    # TODO: COLOR ONLY
    649: ('burn', 'chill', 'douse', 'shock'),  # Genesect
    658: 'ash',  # Greninja
    # TODO: COLOR ONLY
    666: (
        'archipelago', 'continental', 'elegant', 'fancy', 'garden', 'high-plains', 
        'icy-snow', 'jungle', 'marine', 'meadow', 'modern', 'monsoon', 
        'ocean', 'poke-ball', 'polar', 'river', 'sandstorm', 'savanna', 'sun', 'tundra',
    ),  # Vivillon
    668: Form.FEMALE,  # Pyroar
    # TODO: COLOR ONLY
    669: no_normal('blue', 'orange', 'red', 'white', 'yellow'),  # Flabébé
    # TODO: COLOR ONLY except 'az'
    670: ('blue', 'orange', 'red', 'white', 'yellow', 'az'),  # Floette
    # TODO: COLOR ONLY
    671: ('blue', 'orange', 'red', 'white', 'yellow'),  # Florges
    676: (
        'dandy', 'debutante', 'diamond', 'heart', 'kabuki', 
        'la-reine', 'matron', 'pharaoh', 'star',
    ),  # Furfrou
    678: Form.FEMALE,  # Meowstic
    681: no_normal('blade', 'shield'),  # Aegislash
    # 710, 711 => size differences don't matter for us
    716: no_normal('active', 'neutral'),  #  Xerneas
    718: no_normal('cell', 'core', '10-percent', '50-percent', 'complete'),  # Zygarde
    719: Form.MEGA,  # Diancie
    720: no_normal('confined', 'unbound'),  # Hoopa
    741: no_normal('baile', 'pau', 'pom-pom', 'sensu'),  # Oricorio
    745: no_normal('dusk', 'midday', 'midnight'),  # Lycanroc
    746: no_normal('solo', 'school'),  # Wishiwashi
    773: (
        'Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 
        'Fire', 'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 
        'Poison', 'Psychic', 'Rock', 'Steel', 'Water',
    ),  # Silvally
    774: no_normal(
        'meteor', 'core',
        *color_only(
            'core-blue', 'core-green', 'core-indigo', 'core-orange', 
            'core-red', 'core-violet', 'core-yellow',
        ), 
    ),  # Minior
    778: no_normal('busted', 'disguised'),  # Mimikyu
    791: 'radiant-sun',  # Solgaleo
    792: 'full-moon',  # Lunala
    800: ('dusk-mane', 'dawn-wings', 'ultra'),  # Necrozma
    # TODO: COLOR ONLY
    801: 'oringal-color',  # Magearna
    802: 'zenith',  # Marshadow
    809: Form.GIGANTAMAX,  # Melmetal
    812: Form.GIGANTAMAX,  # Rillaboom
    815: Form.GIGANTAMAX,  # Cinderace
    818: Form.GIGANTAMAX,  # Inteleon
    823: Form.GIGANTAMAX,  # Corviknight
    826: Form.GIGANTAMAX,  # Orbeetle
    834: Form.GIGANTAMAX,  # Drednaw
    839: Form.GIGANTAMAX,  # Coalossal
    841: Form.GIGANTAMAX,  # Flapple
    842: Form.GIGANTAMAX,  # Appletun
    844: Form.GIGANTAMAX,  # Sandaconda
    845: ('gorging', 'gulping'),  # Cramorant
    849: no_normal(Form.GIGANTAMAX, 'amped', 'low-key'),  # Toxtricity
    851: Form.GIGANTAMAX,  # Centiskorch
    854: no_normal('antique', 'phony'),  # Sinistea
    855: no_normal('antique', 'phony'),  # Polteageist
    858: Form.GIGANTAMAX,  # Hatterene
    861: Form.GIGANTAMAX,  # Grimmsnarl
    869: no_normal(
        Form.GIGANTAMAX,
        # TODO: COLOR ONLY
        *[
            f'{cream}-{sweet}' for cream, sweet in itertools.product(
                [
                    'vanilla-cream', 'ruby-cream', 'matcha-cream', 'mint-cream', 'lemon-cream', 
                    'salted-cream', 'ruby-swirl', 'caramel-swirl', 'rainbow-swirl',
                ],
                [
                    'strawberry-sweet', 'love-sweet', 'berry-sweet', 'clover-sweet', 'flower-sweet', 
                    'star-sweet', 'ribbon-sweet',
                ],
            )
        ],
    ),  # Alcremie
    875: no_normal('ice-face', 'noice-face'),  # Eiscue
    877: no_normal('full-belly', 'hangry'),  # Morpeko
    879: Form.GIGANTAMAX,  # Copperajah
    884: Form.GIGANTAMAX,  # Duraludon
    888: no_normal('hero-of-many-battles', 'crowned-sword'),  # Zacian
    889: no_normal('hero-of-many-battles', 'crowned-shield'),  # Zamazenta
    890: 'eternamax',  # Eternatus
    892: no_normal(Form.GIGANTAMAX, 'single-strike', 'rapid-strike'),  # Urshufi
    893: 'dada',  # Zarude
    898: ('ice-rider', 'shadow-rider'),  # Calyrex
})
import pprint; pprint.pprint(POKEMON_FORMS)

{1: (PokemonForm(ndex=1, name='__normal__', color_only=False),),
 2: (PokemonForm(ndex=2, name='__normal__', color_only=False),),
 3: (PokemonForm(ndex=3, name='__normal__', color_only=False),
     PokemonForm(ndex=3, name='mega', color_only=False),
     PokemonForm(ndex=3, name='gigantamax', color_only=False)),
 4: (PokemonForm(ndex=4, name='__normal__', color_only=False),),
 5: (PokemonForm(ndex=5, name='__normal__', color_only=False),),
 6: (PokemonForm(ndex=6, name='__normal__', color_only=False),
     PokemonForm(ndex=6, name='mega-x', color_only=False),
     PokemonForm(ndex=6, name='mega-y', color_only=False),
     PokemonForm(ndex=6, name='gigantamax', color_only=False)),
 7: (PokemonForm(ndex=7, name='__normal__', color_only=False),),
 8: (PokemonForm(ndex=8, name='__normal__', color_only=False),),
 9: (PokemonForm(ndex=9, name='__normal__', color_only=False),
     PokemonForm(ndex=9, name='mega', color_only=False),
     PokemonForm(ndex=9, name='gigantamax', color_only=False)

 526: (PokemonForm(ndex=526, name='__normal__', color_only=False),),
 527: (PokemonForm(ndex=527, name='__normal__', color_only=False),),
 528: (PokemonForm(ndex=528, name='__normal__', color_only=False),),
 529: (PokemonForm(ndex=529, name='__normal__', color_only=False),),
 530: (PokemonForm(ndex=530, name='__normal__', color_only=False),),
 531: (PokemonForm(ndex=531, name='__normal__', color_only=False),
       PokemonForm(ndex=531, name='mega', color_only=False)),
 532: (PokemonForm(ndex=532, name='__normal__', color_only=False),),
 533: (PokemonForm(ndex=533, name='__normal__', color_only=False),),
 534: (PokemonForm(ndex=534, name='__normal__', color_only=False),),
 535: (PokemonForm(ndex=535, name='__normal__', color_only=False),),
 536: (PokemonForm(ndex=536, name='__normal__', color_only=False),),
 537: (PokemonForm(ndex=537, name='__normal__', color_only=False),),
 538: (PokemonForm(ndex=538, name='__normal__', color_only=False),),
 539: (PokemonForm(ndex=539, name='__normal

 782: (PokemonForm(ndex=782, name='__normal__', color_only=False),),
 783: (PokemonForm(ndex=783, name='__normal__', color_only=False),),
 784: (PokemonForm(ndex=784, name='__normal__', color_only=False),),
 785: (PokemonForm(ndex=785, name='__normal__', color_only=False),),
 786: (PokemonForm(ndex=786, name='__normal__', color_only=False),),
 787: (PokemonForm(ndex=787, name='__normal__', color_only=False),),
 788: (PokemonForm(ndex=788, name='__normal__', color_only=False),),
 789: (PokemonForm(ndex=789, name='__normal__', color_only=False),),
 790: (PokemonForm(ndex=790, name='__normal__', color_only=False),),
 791: (PokemonForm(ndex=791, name='__normal__', color_only=False),
       PokemonForm(ndex=791, name='radiant-sun', color_only=False)),
 792: (PokemonForm(ndex=792, name='__normal__', color_only=False),
       PokemonForm(ndex=792, name='full-moon', color_only=False)),
 793: (PokemonForm(ndex=793, name='__normal__', color_only=False),),
 794: (PokemonForm(ndex=794, name='__nor

In [2]:
from abc import ABC, abstractmethod
import hashlib
from pathlib import Path
import shutil
from typing import Dict, Iterable, Optional

from cairosvg import svg2png
from py7zr import unpack_7zarchive
import requests
from wand.image import Image


TMP_DIR = Path('../tmp/')


def with_stem(path, stem):
    """https://github.com/python/cpython/blob/56c1f6d7edad454f382d3ecb8cdcff24ac898a50/Lib/pathlib.py#L764-L766"""
    return path.with_name(stem + path.suffix)


class PathDict(dict):
    def __getitem__(self, path: Path):
        assert isinstance(path, Path), f'expected key to be a Path but got {type(path).__name__}'
        for key, val in self.items():
            if path.match(key) or path.with_suffix('').match(key):
                return val
        return None


class DataSource(ABC):
    name = None
    checksum = None
    extra_ops = ()
    
    _filenames = set()
    
    def __init__(self, *, name=None, checksum=None, extra_ops=None):
        if name is not None:
            self.name = name
        assert self.name is not None, 'name is not defined'
        if checksum is not None:
            self.checksum = checksum
        if extra_ops is not None:
            self.extra_ops = extra_ops
    
    def run(self, force=False):
        self.target_dir.mkdir(parents=True, exist_ok=True)
        data_path = self.get(force)
        self.verify_checksum(data_path)
        self.process(data_path)
        self.arrange()
        self.do_extra_ops()
        self.rename_forms()
        self.delete_unused_files()

    @abstractmethod
    def get(self, force):
        ...
    
    def verify_checksum(self, data_path):
        CHUNK_SIZE = 1024 * 64
        hash_sha256 = hashlib.sha256()
        with open(data_path, 'rb') as f:
            for chunk in iter(lambda: f.read(CHUNK_SIZE), b''):
                hash_sha256.update(chunk)

        checksum = hash_sha256.hexdigest()
        if self.checksum is not None:
            assert self.checksum == checksum, (
                f'invalid checksum. expected {self.checksum} but got {checksum}'
            )
        else:
            print(f'checksum for {data_path} is {checksum}')
    
    def process(self, archive):
        ...
    
    @property
    def target_dir(self):
        return TMP_DIR / self.name
    
    def arrange(self):
        ...
    
    def do_extra_ops(self):
        for src, dst in self.extra_ops:
            print('extra_op', src, dst)
            if dst is None:
                try:
                    if (TMP_DIR / src).is_dir():
                        shutil.rmtree(TMP_DIR / src)
                    else:
                        (TMP_DIR / src).unlink()
                except FileNotFoundError:
                    print('tried to delete', TMP_DIR / src)
            else:
                shutil.move(str(TMP_DIR / src), str(TMP_DIR / dst))
    
    def rename_forms(self) -> None:
        """Also stores all used filenames in `self._filenames`."""
        
        self._filenames = set()

        assigned_forms = self.assign_forms()
        for filename in self.get_files():
            stem = filename.stem
            form = assigned_forms[filename]
            if form is None:
                ndex = self.parse_ndex(stem)
                forms = [
                    form 
                    for form in POKEMON_FORMS[ndex] 
                    if form.complete_name == stem
                ]
                assert len(forms) == 1, f'got {len(forms)} matching forms instead 1 for {filename}'
                form = forms[0]
            
            if form is DISMISS_FORM:
                print('dismissing', filename)
                filename.unlink()
            else:
                
                # Avoid unncessary renames
                if form.complete_name != stem:
                    # print('rename:', filename, filename.parent / f'{form.complete_name}{filename.suffix}')
                    # filename.rename(filename.parent / f'{form.complete_name}{filename.suffix}')
                    rename_to = with_stem(filename, form.complete_name)
                    print('rename:', filename, rename_to)
                    filename.rename(rename_to)
                    self._filenames.add(rename_to)
                else:
                    self._filenames.add(filename)

    def assign_forms(self) -> PathDict:
        return PathDict()
    
    @abstractmethod
    def get_files(self) -> Iterable[Path]:
        ...
    
    def parse_ndex(self, filename: str) -> int:
        ndex_str, *rest = dename(filename)
        return int(ndex_str)
    
    @property
    def filenames(self):
        return self._filenames.copy()
    
    def delete_unused_files(self):
        ...


class ArchiveDataSource(DataSource):
    
    def process(self, archive):
        shutil.unpack_archive(filename=archive, extract_dir=self.target_dir)

        
class RemoteDataSource(ArchiveDataSource):
    url = None
    
    def __init__(self, *, url=None, **kwargs):
        super().__init__(**kwargs)
        if url is not None:
            self.url = url
        assert self.url is not None, 'url is not defined'
    
    def get(self, force):
        download_dest = TMP_DIR / Path(self.url).name
#         print(download_dest, download_dest.resolve())
#         print(download_dest.exists(), force)
        if not download_dest.exists() or force:
            print('fetching', self.url)
            response = requests.get(self.url)
            response.raise_for_status()
            with open(download_dest, 'wb') as f:
                f.write(response.content)
        return download_dest


class VeekunDataSource(RemoteDataSource):
    path_to_sprites = Path('pokemon') / 'main-sprites'
    
    def __init__(self, *, path_to_sprites=None, **kwargs):
        super().__init__(**kwargs)
        if path_to_sprites is not None:
            self.path_to_sprites = path_to_sprites
        assert self.path_to_sprites is not None, 'path_to_sprites is not defined'
    
    def arrange(self):
        for p in (self.target_dir / self.path_to_sprites).iterdir():
            if (TMP_DIR / p.name).exists():
                print('deleting existing', TMP_DIR / p.name)
                shutil.rmtree(TMP_DIR / p.name)
            shutil.move(str(p), str(TMP_DIR))  # NOTE: path-like objects are supported since python 3.9
        shutil.rmtree(self.target_dir)


class DreamWorldDataSource(VeekunDataSource):
    
    def run(self, force=False):
        super().run(force)
        self.svg2png()
    
    def svg2png(self):
        for filename in (TMP_DIR / 'dream-world').iterdir():
            with open(filename) as f:
                svg2png(
                    file_obj=f,
                    write_to=str(TMP_DIR / 'dream-world' / f'{filename.stem}.png'),
                )
            filename.unlink()


class BattlersDataSource(RemoteDataSource):
    TARGET_PATH = TMP_DIR / '3d-battlers-animated'
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        try:
            shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)
        except shutil.RegistryError as e:
            if '.7z is already registered for "7zip"' in str(e):
                pass
            else:
                raise
    
    def run(self, force=False):
        super().run(force)
        self.animations2frames()
    
    def get(self, force):
        # TODO: Implement automatic download
        if not (TMP_DIR / '3D Battlers [All].7z').exists():
            print('Please manually download the file from')
            print('  https://www.mediafire.com/folder/mi31mvoxx98ij/3D_Battlers')
            print('and save it as tmp/3D Battlers [All].7z')
        return TMP_DIR / '3D Battlers [All].7z'
        
    def arrange(self):
        if self.TARGET_PATH.exists():
            print('deleting existing', self.TARGET_PATH)
            shutil.rmtree(self.TARGET_PATH)
        # move all files in 'Front' into tmp/anim-3d-battlers
        shutil.move(str(self.target_dir / 'Front'), str(self.TARGET_PATH))
        shutil.rmtree(self.target_dir)
        
        # Remove leading zeros.
        DELIMITER = '_'
        for filename in self.TARGET_PATH.glob('*.png'):
            if DELIMITER in filename.stem:
                padding_num_str, rest = filename.stem.split(DELIMITER, maxsplit=1)
                rest = DELIMITER + rest
            else:
                padding_num_str, rest = filename.stem, ''
            filename.rename(
                self.TARGET_PATH 
                / f'{int(padding_num_str)}{rest}{filename.suffix}'
            )
        
        self.rename_mega_evolutions()
        # TODO: More renamings of forms (i.e. unown but all remaining underscores)
    
    def rename_mega_evolutions(self):
        for num in MEGA_EVOLUTIONS:
            # 2 mega evolutions (X/Y)
            if num in (6, 150):
                old_x = self.TARGET_PATH / f'{num}_1.png'
                old_y = self.TARGET_PATH / f'{num}_2.png'
                assert old_x.exists() and old_y.exists(), (
                    f'invalid filenames {old_x} and {old_y} for mega evolutions'
                )
                old_x.rename(self.TARGET_PATH / f'{num}-mega-x.png')
                old_y.rename(self.TARGET_PATH / f'{num}-mega-y.png')
            # 1 mega evolution
            else:
                old_filename = self.TARGET_PATH / f'{num}_1.png'
                assert old_filename.exists(), f'invalid filename {old_filename} for mega evolution'
                old_filename.rename(self.TARGET_PATH / f'{num}-mega.png')
    
    def animations2frames(self):
        for filename in self.TARGET_PATH.iterdir():
            if filename.stem != '110':
                continue
            print('animations2frames', filename)
            img = Image(filename=filename)
            assert (img.width / img.height).is_integer(), 'invalid/non-integer image ratio'
            frame_width, frame_height = img.height, img.height
            
            i = 0
            x = 0
            first_frame = None
            frames = []
            while x < img.width:
                frame = img[x:x+frame_width, 0:frame_height]
                if x == 0:
                    first_frame = frame
                else:
                    if frame == first_frame:
                        print('found cycle at frame =', i)
                        break
                frames.append(frame)
                frame.format = 'png'
                frame.save(filename=filename.parent / f'{filename.stem}-{i}.png')
                i += 1
                x += frame_width

            filename.unlink()
            
            break


In [ ]:
class Gen1Veekun(VeekunDataSource):
    name = 'generation-1'
    url = 'https://veekun.com/static/pokedex/downloads/generation-1.tar.gz'
    checksum = '2d0923f5abf1171b7e011b3ce9b879e8eee1fd56ec82dfbe597a2eafa63ca21c'
    extra_ops = (
        ('red-blue/back', None),
        ('red-blue/gray', None),
        ('red-green/back', None),
        ('red-green/gray', None),
        ('yellow/back', None),
        ('yellow/gray', None),
        ('yellow-gbc', None),
        ('yellow/gbc', 'yellow-gbc'),
    )
    
    def get_files(self):
        yield from (TMP_DIR / 'red-blue').glob('*.png')
        yield from (TMP_DIR / 'red-green').glob('*.png')
        yield from (TMP_DIR / 'yellow').glob('*.png')
        yield from (TMP_DIR / 'yellow-gbc').glob('*.png')


Gen1Veekun().run()

In [ ]:
class Gen2Veekun(VeekunDataSource):
    name = 'generation-2'
    url = 'https://veekun.com/static/pokedex/downloads/generation-2.tar.gz'
    checksum = '1a01266008cf726df5d273da96ec3cbbbd3da0f17bfada4b0b153a4c92b4517a'
    extra_ops = (
        ('gold/back', None),
        ('gold/shiny', None),
        ('silver/back', None),
        ('silver/shiny', None),
        ('crystal/back', None),
        ('crystal/shiny', None),
        ('crystal/animated/shiny', None),
        ('crystal-animated', None),  # make sure crystal-animated doesn't exist before move
        ('crystal/animated', 'crystal-animated'),
    )
    
    def get_files(self):
        yield from (TMP_DIR / 'gold').glob('*.png')
        yield from (TMP_DIR / 'silver').glob('*.png')
        yield from (TMP_DIR / 'crystal').glob('*.png')
        yield from (TMP_DIR / 'crystal-animated').glob('*.gif')
    
    def assign_forms(self):
        return PathDict(**{
            'gold/201': DISMISS_FORM,
            'silver/201': DISMISS_FORM,
            'crystal/201': DISMISS_FORM,
            'crystal-animated/201': get_form(201, 'u'),
        })  


Gen2Veekun().run()

In [ ]:
class Gen3Veekun(VeekunDataSource):
    name = 'generation-3'
    url = 'https://veekun.com/static/pokedex/downloads/generation-3.tar.gz'
    checksum = '15b733baf9ef91fbde3ae957edb4d2ba75615601a515b41590ab87043370319c'
    extra_ops = (
        ('ruby-sapphire/back', None),
        ('ruby-sapphire/shiny', None),
        ('emerald/animated/shiny', None),
        ('emerald-animated', None),  # make sure emerald-animated doesn't exist before move
        ('emerald/animated', 'emerald-animated'),
        ('emerald-frame2', None),  # make sure emerald-frame2 doesn't exist before move
        ('emerald/frame2', 'emerald-frame2'),
        ('emerald/shiny', None),
        ('firered-leafgreen/back', None),
        ('firered-leafgreen/shiny', None),
    )
    
    def get_files(self):
        yield from (TMP_DIR / 'ruby-sapphire').glob('*.png')
        yield from (TMP_DIR / 'emerald').glob('*.png')
        yield from (TMP_DIR / 'emerald-animated').glob('*.gif')
        yield from (TMP_DIR / 'emerald-frame2').glob('*.png')
        yield from (TMP_DIR / 'firered-leafgreen').glob('*.png')
    
    def assign_forms(self):
        return PathDict(**{
            'ruby-sapphire/201': get_form(201, 'j'),
            'ruby-sapphire/386-normal': get_form(386, Form.NORMAL),
            'emerald/201': get_form(201, 'j'),
            'emerald/386-normal': get_form(386, Form.NORMAL),
            'emerald-animated/386-normal': get_form(386, Form.NORMAL),
            'emerald-frame2/201*': DISMISS_FORM,  # empty image
            'emerald-frame2/351*': DISMISS_FORM,  # empty image
            'emerald-frame2/386*': DISMISS_FORM,  # empty image
            'firered-leafgreen/386-normal': get_form(386, Form.NORMAL),
        })


Gen3Veekun().run()

In [ ]:
class Gen4Veekun(VeekunDataSource):
    name = 'generation-4'
    url = 'https://veekun.com/static/pokedex/downloads/generation-4.tar.gz'
    checksum = 'b1b69463aac872b54adf56f1159e8e6d2dfcbbecb7d71c7ebf832fe44140da41'
    extra_ops = (
        ('diamond-pearl/back', None),
        ('diamond-pearl/female', None),
        ('diamond-pearl-frame2', None),  # make sure diamond-pearl-frame2 doesn't exist before move
        ('diamond-pearl/frame2', 'diamond-pearl-frame2'),
        ('diamond-pearl/shiny', None),
        ('platinum/back', None),
        ('platinum/female', None),
        ('platinum-frame2', None),  # make sure platinum-frame2 doesn't exist before move
        ('platinum/frame2', 'platinum-frame2'),
        ('platinum/shiny', None),
        ('heartgold-soulsilver/back', None),
        ('heartgold-soulsilver/female', None),
        ('heartgold-soulsilver-frame2', None),  # make sure heartgold-soulsilver-frame2 doesn't exist before move
        ('heartgold-soulsilver/frame2', 'heartgold-soulsilver-frame2'),
        ('heartgold-soulsilver/shiny', None),
    )
    
    def get_files(self):
        yield from (TMP_DIR / 'diamond-pearl').glob('*.png')
        yield from (TMP_DIR / 'diamond-pearl-frame2').glob('*.png')
        yield from (TMP_DIR / 'platinum').glob('*.png')
        yield from (TMP_DIR / 'platinum-frame2').glob('*.png')
        yield from (TMP_DIR / 'heartgold-soulsilver').glob('*.png')
        yield from (TMP_DIR / 'heartgold-soulsilver-frame2').glob('*.png')
    
    def assign_forms(self):
        return PathDict(**{
            'diamond-pearl/201': get_form(201, 'a'),
            'diamond-pearl/386-normal': get_form(386, Form.NORMAL),
            'diamond-pearl/412': get_form(412, 'plant'),
            'diamond-pearl/413': get_form(413, 'plant'),
            'diamond-pearl/421': get_form(421, 'overcast'),
            'diamond-pearl/422': get_form(422, 'west'),
            'diamond-pearl/423': get_form(423, 'west'),
            'diamond-pearl/487': get_form(487, 'altered'),
            'diamond-pearl/492': get_form(492, 'land'),
            'diamond-pearl/493-normal': get_form(493, Form.NORMAL),
            'diamond-pearl/493-unknown': DISMISS_FORM,  # invalid form
            'diamond-pearl-frame2/201': DISMISS_FORM,  # empty image
            'diamond-pearl-frame2/386-normal': get_form(386, Form.NORMAL),
            'diamond-pearl-frame2/412': get_form(412, 'plant'),
            'diamond-pearl-frame2/413': get_form(413, 'plant'),
            'diamond-pearl-frame2/421': get_form(421, 'overcast'),
            'diamond-pearl-frame2/422': DISMISS_FORM,  # invalid form, seems mixed
            'diamond-pearl-frame2/423': DISMISS_FORM,  # invalid form, seems mixed
            'diamond-pearl-frame2/487': get_form(487, 'altered'),
            'diamond-pearl-frame2/492': get_form(492, 'land'),
            'diamond-pearl-frame2/493-normal': get_form(493, Form.NORMAL),
            'diamond-pearl-frame2/493-unknown': DISMISS_FORM,  # invalid form
            'platinum/201': get_form(201, 'a'),
            'platinum/386-normal': get_form(386, Form.NORMAL),
            'platinum/412': get_form(412, 'plant'),
            'platinum/413': get_form(413, 'plant'),
            'platinum/421': get_form(421, 'overcast'),
            'platinum/422': get_form(422, 'west'),
            'platinum/423': get_form(423, 'west'),
            'platinum/487': get_form(487, 'altered'),
            'platinum/492': get_form(492, 'land'),
            'platinum/493-normal': get_form(493, Form.NORMAL),
            'platinum/493-unknown': DISMISS_FORM,  # invalid form
            'platinum-frame2/201*': DISMISS_FORM,  # empty image
            'platinum-frame2/351*': DISMISS_FORM,  # empty image
            'platinum-frame2/386*': DISMISS_FORM,  # empty image
            'platinum-frame2/412': get_form(412, 'plant'),
            'platinum-frame2/413': get_form(413, 'plant'),
            'platinum-frame2/421': get_form(421, 'overcast'),
            'platinum-frame2/422': get_form(422, 'west'),
            'platinum-frame2/423': get_form(423, 'west'),
            'platinum-frame2/487': get_form(487, 'altered'),
            'platinum-frame2/492': get_form(492, 'land'),
            'platinum-frame2/493-normal': get_form(493, Form.NORMAL),
            'platinum-frame2/493-unknown': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver/172-beta': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver/201': get_form(201, 'a'),
            'heartgold-soulsilver/386-normal': get_form(386, Form.NORMAL),
            'heartgold-soulsilver/412': get_form(412, 'plant'),
            'heartgold-soulsilver/412-beta': DISMISS_FORM,
            'heartgold-soulsilver/413': get_form(413, 'plant'),
            'heartgold-soulsilver/421': get_form(421, 'overcast'),
            'heartgold-soulsilver/421-beta': DISMISS_FORM,
            'heartgold-soulsilver/422': get_form(422, 'west'),  # different animation frame than 422-west
            'heartgold-soulsilver/423': get_form(423, 'west'),  # different animation frame than 423-west
            'heartgold-soulsilver/487': get_form(487, 'altered'),
            'heartgold-soulsilver/492': get_form(492, 'land'),
            'heartgold-soulsilver/493-normal': get_form(493, Form.NORMAL),
            'heartgold-soulsilver/493-unknown': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver/egg': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver/egg-manaphy': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver/substitute': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver-frame2/201*': DISMISS_FORM,  # empty image
            'heartgold-soulsilver-frame2/386-normal': get_form(386, Form.NORMAL),
            'heartgold-soulsilver-frame2/412': get_form(412, 'plant'),
            'heartgold-soulsilver-frame2/413': get_form(413, 'plant'),
            'heartgold-soulsilver-frame2/421': get_form(421, 'overcast'),
            'heartgold-soulsilver-frame2/422': get_form(422, 'west'),
            'heartgold-soulsilver-frame2/423': get_form(423, 'west'),
            'heartgold-soulsilver-frame2/487': get_form(487, 'altered'),
            'heartgold-soulsilver-frame2/492': get_form(492, 'land'),
            'heartgold-soulsilver-frame2/493-normal': get_form(493, Form.NORMAL),
            'heartgold-soulsilver-frame2/493-unknown': DISMISS_FORM,  # invalid form
            'heartgold-soulsilver-frame2/egg': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver-frame2/egg-manaphy': DISMISS_FORM,  # no pokemon
            'heartgold-soulsilver-frame2/substitute': DISMISS_FORM,  # no pokemon
        })


Gen4Veekun().run()

In [ ]:
class Gen5Veekun(VeekunDataSource):
    name = 'generation-5'
    url = 'https://veekun.com/static/pokedex/downloads/generation-5.tar.gz'
    checksum = 'ee037a3319b2a6143c5c90f679be13a06126c2f5424e46023fe0f53d2631aa62'
    extra_ops = (
        ('black-white/back', None),
        ('black-white/female/521.png', 'black-white/521-female.png'),
        ('black-white/female/592.png', 'black-white/592-female.png'),
        ('black-white/female/593.png', 'black-white/593-female.png'),
        ('black-white/female', None),
        ('black-white/shiny', None),
    )
    
    def get_files(self):
        yield from (TMP_DIR / 'black-white').glob('*.png')
    
    def assign_forms(self):
        return PathDict(**{
            'black-white/0': DISMISS_FORM,  # no pokemon
            'black-white/201': get_form(201, 'a'),
            'black-white/386-normal': get_form(386, Form.NORMAL),
            'black-white/412': get_form(412, 'plant'),
            'black-white/413': get_form(413, 'plant'),
            'black-white/421': get_form(421, 'overcast'),
            'black-white/422': get_form(422, 'west'),
            'black-white/423': get_form(423, 'west'),
            'black-white/487': get_form(487, 'altered'),
            'black-white/492': get_form(492, 'land'),
            'black-white/493-normal': get_form(493, Form.NORMAL),
            'black-white/550': get_form(550, 'red-striped'),
            'black-white/555': get_form(555, Form.NORMAL),
            'black-white/555-standard': get_form(555, Form.NORMAL),
            'black-white/555-zen': get_form(555, 'zen'),
            'black-white/585': get_form(585, 'spring'),
            'black-white/586': get_form(586, 'spring'),
            'black-white/641': get_form(641, 'incarnate'),
            'black-white/642': get_form(642, 'incarnate'),
            'black-white/645': get_form(645, 'incarnate'),
            'black-white/647': get_form(647, 'ordinary'),
            'black-white/648': get_form(648, 'aria'),
            'black-white/female/521': get_form(521, Form.FEMALE),
            'black-white/female/592': get_form(592, Form.FEMALE),
            'black-white/female/593': get_form(593, Form.FEMALE),
            'black-white/egg': DISMISS_FORM,  # no pokemon
            'black-white/egg-manaphy': DISMISS_FORM,  # no pokemon
            'black-white/substitute': DISMISS_FORM,  # no pokemon
        })


gen5_veekun = Gen5Veekun()
gen5_veekun.run()
# import pprint; pprint.pprint(gen5_veekun.filenames)

In [ ]:
class IconsVeekun(VeekunDataSource):
    name = 'icons-unpacked'
    url = 'https://veekun.com/static/pokedex/downloads/pokemon-icons.tar.gz'
    checksum = 'f9850ce82d8e6e69c163112c47553458fd27805034217a5331a1ae12b2a1c8ac'
    path_to_sprites = Path('pokemon')
    extra_ops = (
        ('icons/egg.png', None),
        ('icons/female/521.png', 'icons/521-female.png'),
        ('icons/female/592.png', 'icons/592-female.png'),
        ('icons/female/593.png', 'icons/593-female.png'),
        ('icons/female/668.png', 'icons/668-female.png'),
        ('icons/female/678.png', 'icons/678-female.png'),
        ('icons/female', None),
        ('icons/old', None),
        ('icons/right', None),
    )
    
    def get_files(self):
        yield from (TMP_DIR / 'icons').glob('*.png')
    
    def assign_forms(self):
        return PathDict(**{
            'icons/25-cosplay': get_form(25, 'cosplay'),
            'icons/25-rock-star': get_form(25, 'cosplay-rock-star'),
            'icons/25-belle': get_form(25, 'cosplay-belle'),
            'icons/25-pop-star': get_form(25, 'cosplay-pop-star'),
            'icons/25-phd': get_form(25, 'cosplay-phd'),
            'icons/25-libre': get_form(25, 'cosplay-libre'),
            'icons/201': get_form(201, 'a'),
            'icons/386-normal': get_form(386, Form.NORMAL),
            'icons/412': get_form(412, 'plant'),
            'icons/413': get_form(413, 'plant'),
            'icons/421': get_form(421, 'overcast'),
            'icons/422': get_form(422, 'west'),
            'icons/423': get_form(423, 'west'),
            'icons/487': get_form(487, 'altered'),
            'icons/492': get_form(492, 'land'),
            'icons/493-*': DISMISS_FORM,  # invalid forms (equal normal)
            'icons/550': get_form(550, 'red-striped'),
            'icons/555': get_form(555, Form.NORMAL),
            'icons/555-standard': get_form(555, Form.NORMAL),
            'icons/555-zen': get_form(555, 'zen'),
            'icons/585': get_form(585, 'spring'),
            'icons/586': get_form(586, 'spring'),
            'icons/641': get_form(641, 'incarnate'),
            'icons/642': get_form(642, 'incarnate'),
            'icons/645': get_form(645, 'incarnate'),
            'icons/647': get_form(647, 'ordinary'),
            'icons/648': get_form(648, 'aria'),
            'icons/649-*': DISMISS_FORM,  # invalid forms (equal normal)
            'icons/666': get_form(666, 'meadow'),
            'icons/669': get_form(669, 'red'),
            'icons/670': get_form(670, 'red'),
            'icons/670-eternal': DISMISS_FORM,  # unknown form
            'icons/671': get_form(671, 'red'),
            'icons/676': get_form(676, Form.NORMAL),
            'icons/676-natural': get_form(676, Form.NORMAL),
            'icons/678-male': get_form(678, Form.NORMAL),  # TODO
            'icons/678-female': get_form(678, Form.FEMALE),
            'icons/681': get_form(681, 'shield'),
            'icons/710-*': DISMISS_FORM,
            'icons/711-*': DISMISS_FORM,
            'icons/716': get_form(716, 'active'),
            'icons/718': get_form(718, '50-percent'),
            'icons/720': get_form(720, 'confined'),
            'icons/egg': DISMISS_FORM,
        })


icons_veekun = IconsVeekun()
icons_veekun.run()

In [167]:
# NOTE: According to 
#  https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_with_gender_differences 
#  significant differences between male and female pokemon 
#  start happening from generation 5:
#  => 521, 592, 593, 668, 678, 876

# gen1_veekun = VeekunDataSource(
#     name='generation-1', 
#     url='https://veekun.com/static/pokedex/downloads/generation-1.tar.gz',
#     checksum='2d0923f5abf1171b7e011b3ce9b879e8eee1fd56ec82dfbe597a2eafa63ca21c',
#     extra_ops=(
#         ('red-blue/back', None),
#         ('red-blue/gray', None),
#         ('red-green/back', None),
#         ('red-green/gray', None),
#         ('yellow/back', None),
#         ('yellow/gray', None),
#         ('yellow/gbc', 'yellow-gbc'),
#     ),
# )
# gen2_veekun = VeekunDataSource(
#     name='generation-2', 
#     url='https://veekun.com/static/pokedex/downloads/generation-2.tar.gz',
#     checksum='1a01266008cf726df5d273da96ec3cbbbd3da0f17bfada4b0b153a4c92b4517a',
#     extra_ops=(
#         ('gold/back', None),
#         ('gold/shiny', None),
#         ('silver/back', None),
#         ('silver/shiny', None),
#         ('crystal/back', None),
#         ('crystal/shiny', None),
#         ('crystal/animated/shiny', None),
#         ('crystal-animated', None),  # make sure diamond-pearl-frame2 doesn't exist before move
#         ('crystal/animated', 'crystal-animated'),
#     ),
# )
# gen3_veekun = VeekunDataSource(
#     name='generation-3', 
#     url='https://veekun.com/static/pokedex/downloads/generation-3.tar.gz',
#     checksum='15b733baf9ef91fbde3ae957edb4d2ba75615601a515b41590ab87043370319c',
#     extra_ops=(
#         ('ruby-sapphire/back', None),
#         ('ruby-sapphire/shiny', None),
#         ('emerald/animated/shiny', None),
#         ('emerald/animated', 'emerald-animated'),
#         ('emerald/frame2/386-normal.png', None),
#         ('emerald/frame2/386-speed.png', None),
#         ('emerald/frame2/386.png', None),
#         ('emerald-frame2', None),  # make sure emerald-frame2 doesn't exist before move
#         ('emerald/frame2', 'emerald-frame2'),
#         ('emerald/shiny', None),
#         ('firered-leafgreen/back', None),
#         ('firered-leafgreen/shiny', None),
#     ),
# )
# gen4_veekun = VeekunDataSource(
#     name='generation-4', 
#     url='https://veekun.com/static/pokedex/downloads/generation-4.tar.gz',
#     checksum='b1b69463aac872b54adf56f1159e8e6d2dfcbbecb7d71c7ebf832fe44140da41',
#     extra_ops=(
#         ('diamond-pearl/back', None),
#         ('diamond-pearl/female', None),
#         ('diamond-pearl-frame2', None),  # make sure diamond-pearl-frame2 doesn't exist before move
#         ('diamond-pearl/frame2', 'diamond-pearl-frame2'),
#         ('diamond-pearl/shiny', None),
#         ('platinum/back', None),
#         ('platinum/female', None),
#         ('platinum-frame2', None),  # make sure diamond-pearl-frame2 doesn't exist before move
#         ('platinum/frame2', 'platinum-frame2'),
#         ('platinum-frame2/frame2', None),
#         ('platinum/shiny', None),
#         ('heartgold-soulsilver/back', None),
#         ('heartgold-soulsilver/egg-manaphy.png', None),
#         ('heartgold-soulsilver/egg.png', None),
#         ('heartgold-soulsilver/female', None),
#         ('heartgold-soulsilver/frame2/egg-manaphy.png', None),
#         ('heartgold-soulsilver/frame2/egg.png', None),
#         ('heartgold-soulsilver/frame2/substitute.png', None),
#         ('heartgold-soulsilver-frame2', None),
#         ('heartgold-soulsilver/frame2', 'heartgold-soulsilver-frame2'),
#         ('heartgold-soulsilver/shiny', None),
#     ),
# )
# gen5_veekun = VeekunDataSource(
#     name='generation-5', 
#     url='https://veekun.com/static/pokedex/downloads/generation-5.tar.gz',
#     checksum='ee037a3319b2a6143c5c90f679be13a06126c2f5424e46023fe0f53d2631aa62',
#     extra_ops=(
#         ('black-white/back', None),
#         ('black-white/egg-manaphy.png', None),
#         ('black-white/egg.png', None),
#         ('black-white/female/521.png', 'black-white/521-female.png'),
#         ('black-white/female/592.png', 'black-white/592-female.png'),
#         ('black-white/female/593.png', 'black-white/593-female.png'),
#         ('black-white/female', None),
#         ('black-white/shiny', None),
#     ),
# )

# icons_veekun = VeekunDataSource(
#     name='icons-unpacked', 
#     url='https://veekun.com/static/pokedex/downloads/pokemon-icons.tar.gz',
#     path_to_sprites=Path('pokemon'),
#     checksum='f9850ce82d8e6e69c163112c47553458fd27805034217a5331a1ae12b2a1c8ac',
#     extra_ops=(
#         ('icons/egg.png', None),
#         ('icons/female/521.png', 'icons/521-female.png'),
#         ('icons/female/592.png', 'icons/592-female.png'),
#         ('icons/female/593.png', 'icons/593-female.png'),
#         ('icons/female/668.png', 'icons/668-female.png'),
#         ('icons/female/678.png', 'icons/678-female.png'),
#         ('icons/female', None),
#         ('icons/old', None),
#         ('icons/right', None),
        
#     ),
# )
sugimori_veekun = VeekunDataSource(
    name='sugimori-unpacked', 
    url='https://veekun.com/static/pokedex/downloads/pokemon-sugimori.tar.gz',
    path_to_sprites=Path('pokemon'),
    checksum='9dcb5ab803725db99ec235df72da9cc20e96ac843d88394cff95a6b0bb06da16',
    extra_ops=(
        ('sugimori/female/521.png', 'sugimori/521-female.png'),
        ('sugimori/female/592.png', 'sugimori/592-female.png'),
        ('sugimori/female/593.png', 'sugimori/593-female.png'),
        ('sugimori/female/668.png', 'sugimori/668-female.png'),
        ('sugimori/female/', None),
    ),
)
dream_world_veekun = DreamWorldDataSource(
    name='dream-world-unpacked', 
    url='https://veekun.com/static/pokedex/downloads/pokemon-dream-world.tar.gz',
    path_to_sprites=Path('pokemon'),
    checksum='eaaf06ea99e71e34d8710f5cfd4923b8cd4d62f44124930afd02bc17046b6057',
    extra_ops=(
        # 521-female not available
        ('dream-world/female/592.svg', 'dream-world/592-female.svg'),
        ('dream-world/female/593.svg', 'dream-world/593-female.svg'),
        ('dream-world/female/', None),
    ),
)

battlers = BattlersDataSource(
    name='3d-battlers-unpacked',
    url='https://www.mediafire.com/folder/mi31mvoxx98ij/3D_Battlers',
    checksum='a282265f827aaf309f08c1be7ea98726de14bca942823ea85e6d7c77338d1205',
    extra_ops=(
        ('3d-battlers-animated/Female/521.png', '3d-battlers-animated/521-female.png'),
        ('3d-battlers-animated/Female/592.png', '3d-battlers-animated/592-female.png'),
        ('3d-battlers-animated/Female/593.png', '3d-battlers-animated/593-female.png'),
        ('3d-battlers-animated/Female/668.png', '3d-battlers-animated/668-female.png'),
        ('3d-battlers-animated/Female/678.png', '3d-battlers-animated/678-female.png'),
        ('3d-battlers-animated/Female/', None),
    ),
)


# gen1_veekun.run()
# gen2_veekun.run()
# gen3_veekun.run()
# gen4_veekun.run()
# gen5_veekun.run()
# icons_veekun.run()
# sugimori_veekun.run()
# dream_world_veekun.run()
battlers.run()

deleting existing ../tmp/3d-battlers-animated
animations2frames ../tmp/3d-battlers-animated/110.png
../tmp/3d-battlers-animated/110.png 239.0
